In [6]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load() 

In [7]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  11


In [9]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n

In [14]:
# from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings

# from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
# from langchain_community import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What kind of services they provide?")

In [16]:
len(retrieved_docs)
print(retrieved_docs[0].page_content)

Icon

Apartment Moving

Efficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.

Icon

Villa Moving

Comprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.

Icon

Household Moving

Full-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.

Icon

Office Moving

Specialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.

Icon

Furniture Moving

Experienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.

Icon

Packing and Unpacking Services

Optional packing and unpacking services available to save you time and effort, using high-quality packing materials.

Icon

Customized Moving Plans


In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [18]:
llm = ChatOpenAI(temperature=0.4, max_tokens=500)

/tmp/ipykernel_485628/244254262.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.4, max_tokens=500)


In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [20]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])

Victoria on Move provides a range of moving services including apartment moving, villa moving, household moving, office moving, and furniture moving. They also offer optional packing and unpacking services using high-quality materials, as well as customized moving plans tailored to fit specific requirements and schedules. Their professional team is experienced and dedicated to delivering exceptional service and customer satisfaction throughout the entire moving process.
